# Урок 3
Это часть 2/2: эксперименты.
## План (напоминание)
В этой практической работе мы узнаем новые трюки и попробуем через эксперименты посмотреть, насколько эти трюки хороши.

В первом ноутбуке мы знакомились с Dropout и Batch Normalization слоями.

**Теперь же** мы вернемся к модели классификации изображений из прошлого урока и попробуем ее улучшать.
А именно:
- настроим сохранение метрик в wandb, обучим бейзлайн и сохраним его метрики;
- добьемся воспроизводимости обучения;
- попробуем применить LR Scheduler, сравним метрики;
- попробуем добавить Dropout и Batch Normalization, сравним метрики;

Помимо этого, мы разберем, как не потерять обученную модель.

## Эксперименты: ставим и сравниваем

#### Подключаем wandb к пайплайну обучения

У меня вознакли сложности с работой wandb в облаке из-за санкуий США.
wandb можно развернуть локально при наличии проблем с онлайн версией
Нужно скачать docker image:
> docker pull wandb/local

Затем запустить сервер
> wandb server start
Команда поднимет контейнер с wandb/local и создаст volume для хранения данных

Для остановки сервера:
> wandb server stop

Нужно сгенерировать apikey на локальном сервере

Можно на deploy.wandb.ai сгенерировать бесплатную лицензию и скопировать ее в localhost:8080/system-admin, чтобы не было предупреждения на главном экране

In [ ]:
import wandb

# with open("../wandb_apikey") as apikey:
#     key = apikey.read()

# print(key)
# wandb.login(key=key,
#             host="http://localhost:8080",
#             relogin=True)
run = wandb.init(project="project-1")  # БЕЗ entity!
run.log({"metric": 1101})
run.finish()

print("✅ Работает!")

In [ ]:
# нужно установить библиотеку wandb
import tqdm
import wandb
import time

# затем можно запускать любой код и логгировать любые метрики

def simple_train_loop():
    # Сначала нужно вызвать wandb.init()
    # Это создаст эксперимент в wandb.ai и привяжет его к текущему запуску.
    run = wandb.init(
        # Более детальное описание аргументов: https://docs.wandb.ai/guides/track/launch
        project="wandb-project",
        notes="I created it in my DL course",
        # Можно так же передать config - словарь с любым содержимым.
        # Обычно туда кладут гиперпараметры, настройки обработки данных, random seed и т.д.
        config={"seed": 0, "my-custom_string": "asb"},
    )
    # Теперь запускаем наш код обучения, подготовки данных и т.п. как обычно
    for i in tqdm.trange(300):
        # Имитируем долгое обучение
        time.sleep(0.01)
        # Нужно добавить эту строку, чтобы записать в wandb
        run.log({"iteration": i, "loss": 10 - i ** 0.3})
    # Запуск автоматически завершится, когда скрипт (т.е. ноутбук) завершит работу.
    # Но можно явно завершить:
    run.finish()

# запустим, смотрим
simple_train_loop()

### Готовим пайплайн классификации

Достанем код из прошлого урока, построим модель, добавим в обучение wandb - и убедимся, что все работает.

In [ ]:
# А теперь делаем все серьезно.
# Загрузим данные, обучим модель, отрисуем в wandb графики
# Дальше идет код с предыдущей лекции
import http.client
import tarfile
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


def prepare_data():
    """Скачивает данные и распаковывает их."""
    target_file = "notMNIST_small.tar.gz"
    if Path(target_file).exists():
        print("Файл уже загружен, не загружаю снова")
    else:
        conn = http.client.HTTPConnection("yaroslavvb.com", 80)
        conn.request("GET", "/upload/notMNIST/notMNIST_small.tar.gz")
        data = conn.getresponse().read()
        with open(target_file, "wb") as f:
            f.write(data)
    with tarfile.open(target_file) as f:
        f.extractall(filter="data")
    print("Данные были скачены и распакованы")


def read_notmnist_data(
    data_dir: str = "notMNIST_small",
) -> tuple[np.ndarray, np.ndarray]:
    """Прочитать картинки датасета notMNIST и положить их в numpy-массив.

    :returns: пару numpy-массивов (изображения, соответствующие метки)
    """
    images, labels = [], []
    for img_path in Path(data_dir).glob("**/*.png"):
        # Имя папки - это метка класса
        img_label = img_path.parts[1]
        try:
            image = plt.imread(img_path)
        except SyntaxError:
            print(
                f"Изображение не читается по пути {img_path} (это ок, но таких должно быть < 10)"
            )
            continue
        labels.append(img_label)
        images.append(image)
    return np.stack(images, axis=0), np.stack(labels, axis=0)


prepare_data()
X, y = read_notmnist_data()
assert X.shape[0] == y.shape[0]
ohe = LabelEncoder()
y = ohe.fit_transform(y)

In [ ]:
import torch
print(torch.__version__, torch.cuda.is_available())

In [ ]:
import torch

seed = 0
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    shuffle=True,
    random_state=1
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val,
    test_size=0.2,
    shuffle=True,
    random_state=1
)
X_train, y_train = torch.from_numpy(X_train), torch.from_numpy(y_train)
X_val, y_val = torch.from_numpy(X_val), torch.from_numpy(y_val)
X_test, y_test = torch.from_numpy(X_test), torch.from_numpy(y_test)

fig, ax = plt.subplots(4, 4, figsize=(8, 8))
for row in range(4):
    for col in range(4):
        idx = 4 * row + col
        ax[row][col].imshow(X_train[idx])
        ax[row][col].set_title(f"label={y_train[idx]}")
# конец кода с предыдущей лекции
# Перезапустите эту ячейку несколько раз - изображения будут меняться.

In [ ]:
import torch.nn as nn


class SimpleModel(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        hidden_dim = 256
        self.net = nn.Sequential(
            nn.Linear(in_features=28 * 28, out_features=hidden_dim),
            nn.ReLU(),
            nn.Linear(in_features=hidden_dim, out_features=num_classes),
            nn.Softmax(dim=1),
        )

    def forward(self, x: torch.Tensor):
        x = x.reshape((-1, 28 * 28))
        return self.net(x)

In [ ]:
import torch.nn.functional as F
from torch.optim.sgd import SGD
from dataclasses import dataclass


# конфиги можно сделать в словаре, а можно в датаклассе - будут подсказки в редакторе
@dataclass
class TrainConfig:
    eval_every: int = 10
    lr: float = 1e-2
    total_iterations: int = 3000


def train_loop(
    model: nn.Module,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    wandb.init(
        project="simple-model-train",
        notes="version 1",
        tags=["sgd", "2-layer"],
        config=config,
    )

    optim = SGD(model.parameters(), lr=config.lr)
    model.train()
    for i in tqdm.trange(config.total_iterations):
        optim.zero_grad()
        loss = F.cross_entropy(model(X_train), y_train)
        loss.backward()
        optim.step()
        metrics = {"iteration": i, "loss_train": loss.detach().cpu().item()}
        # каждые `eval_every` итераций будем считать метрику на отложенной выборке
        if (i + 1) % config.eval_every == 0:
            with torch.no_grad():
                model.eval()
                loss_val = F.cross_entropy(model(X_val), y_val)
                model.train()
                metrics.update({"loss_val": loss_val.cpu().item()})
        wandb.log(metrics)
    wandb.finish()


torch.random.manual_seed(seed)
config = TrainConfig(eval_every=20, lr=1e-1, total_iterations=500)
model = SimpleModel(num_classes=len(ohe.classes_))
train_loop(model, X_train, y_train, X_val, y_val, config=config)

In [ ]:
# попробуем то же самое на GPU

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Используем", device)

def train_loop_device(
    model: nn.Module,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    wandb.init(
        project="simple-model-train",
        notes="version 1",
        tags=["sgd", "2-layer"],
        config=config,
    )

    X_val_dev = X_val.clone().to(device)
    y_val_dev = y_val.clone().to(device)

    optim = SGD(model.parameters(), lr=config.lr)
    # Перенесем на GPU если возможно
    model = model.to(device)
    model.train()
    for i in tqdm.trange(config.total_iterations):
        optim.zero_grad()

        X_train_dev = X_train.clone().to(device)
        y_train_dev = y_train.clone().to(device)

        loss = F.cross_entropy(model(X_train_dev), y_train_dev)
        loss.backward()
        optim.step()
        metrics = {"iteration": i, "loss_train": loss.detach().cpu().item()}
        # каждые `eval_every` итераций будем считать метрику на отложенной выборке
        if (i + 1) % config.eval_every == 0:
            with torch.no_grad():
                model.eval()
                loss_val = F.cross_entropy(model(X_val_dev), y_val_dev)
                model.train()
                metrics.update({"loss_val": loss_val.cpu().item()})
        wandb.log(metrics)
    wandb.finish()


torch.random.manual_seed(seed)
config = TrainConfig(eval_every=20, lr=1e-1, total_iterations=500)
model = SimpleModel(num_classes=len(ohe.classes_))
train_loop_device(model, X_train, y_train, X_val, y_val, config=config)

Если перезапустить ноутбук, то графики получатся другие.

Но как же так, мы ничего не меняли в коде?

## Добиваемся воспроизводимости
Пройдемся по ноутбуку и найдем все места, где есть случайности:
- разбиение на train/val/test - это видно по `shuffle=True`;
- инициализация весов модели - веса всех слоев генерируются из случайного распределения;
- могут быть алгоритмы внутри слоев, но у нас таких нету.

Чтобы добиться воспроизводимости, нужно:
1. Добавить `random_state=...` в `train_test_split` - он умеет принимать такой параметр.
2. Зафиксировать `seed` у PyTorch **перед** созданием модели через `torch.random.manual_seed`.

Идем и исправляем это, после чего перезапускаем ячейки.
Теперь графики перестали меняться от запуска к запуску.

## Пробуем новые подходы

### Добавляем LR Scheduler и Adam
Добавим шедулер, его настройки в конфиг, а также переберем несколько вариантов.

Аналогично сделаем для оптимайзера: будем пробовать Adam и SGD.

In [ ]:
import typing as tp

from torch.optim.lr_scheduler import ExponentialLR, LinearLR, StepLR
from torch.optim.adam import Adam


# конфиги можно сделать в словаре, а можно в датаклассе - будут подсказки в редакторе
@dataclass
class TrainConfig:
    eval_every: int = 10
    lr: float = 1e-2
    total_iterations: int = 3000
    scheduler_type: tp.Literal["exp", "linear", "step"] | None = None
    optimizer_type: tp.Literal["sgd", "adam"] = "sgd"


def train_loop(
    model: nn.Module,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
    run_name: str | None = None,
):
    wandb.init(
        project="simple-model-train",
        notes="version 1",
        # еще добавим возможность называть запуски
        name=run_name,
        tags=[config.optimizer_type, str(config.scheduler_type)],
        config=config,
    )

    #### Новое: создаем разный optimizer в зависимости от конфига ####
    if config.optimizer_type == "sgd":
        optim = SGD(model.parameters(), lr=config.lr)
    else:
        optim = Adam(model.parameters(), lr=config.lr)
    #### Новое: создаем шедулер ####
    # LR scheduler на вход принимает optimizer и некоторые параметры (которые зависят от его алгоритма)
    if config.scheduler_type == "exp":
        scheduler = ExponentialLR(optim, gamma=0.99)
    elif config.scheduler_type == "linear":
        scheduler = LinearLR(
            optim, start_factor=1.0, end_factor=0.1, total_iters=config.total_iterations
        )
    elif config.scheduler_type == "step":
        scheduler = StepLR(optim, step_size=10, gamma=0.9)
    else:
        scheduler = None
    #####

    model.train()
    for i in tqdm.trange(config.total_iterations):
        optim.zero_grad()
        loss = F.cross_entropy(model(X_train), y_train)
        loss.backward()
        optim.step()
        metrics = {"iteration": i, "loss_train": loss.detach().cpu().item()}
        # каждые `eval_every` итераций будем считать метрику на отложенной выборке
        if (i + 1) % config.eval_every == 0:
            with torch.no_grad():
                model.eval()
                loss_val = F.cross_entropy(model(X_val), y_val)
                model.train()
                metrics.update({"loss_val": loss_val.cpu().item()})
        if scheduler is not None:
            # для scheduler точно так же надо звать .step(), но после обучения и валидации
            # см. пример в документации: https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html
            scheduler.step()
            metrics.update({"lr": scheduler.get_last_lr()[0]})
        else:
            # Чтобы иметь одинаковый набор графиков
            metrics.update({"lr": config.lr})
        wandb.log(metrics)
    wandb.finish()
    return optim


# на exp быстрее к нулю сойдемся, без scheduler тогда ок.
for optim in ("sgd", "adam"):
    for scheduler_type in (None, "exp", "linear", "step"):
        torch.random.manual_seed(seed)
        config = TrainConfig(
            eval_every=20,
            lr=2,
            total_iterations=500,
            scheduler_type=scheduler_type,
            optimizer_type=optim,
        )
        model = SimpleModel(num_classes=len(ohe.classes_))
        train_loop(
            model,
            X_train,
            y_train,
            X_val,
            y_val,
            config=config,
            run_name=f"optim={optim}__lr_sched={scheduler_type}",
        )

In [ ]:
# Попробуем то же самое на GPU

In [ ]:
import typing as tp

from torch.optim.lr_scheduler import ExponentialLR, LinearLR, StepLR
from torch.optim.adam import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Используем", device)

# конфиги можно сделать в словаре, а можно в датаклассе - будут подсказки в редакторе
@dataclass
class TrainConfig:
    eval_every: int = 10
    lr: float = 1e-2
    total_iterations: int = 3000
    scheduler_type: tp.Literal["exp", "linear", "step"] | None = None
    optimizer_type: tp.Literal["sgd", "adam"] = "sgd"


def train_loop_dev(
    model: nn.Module,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
    run_name: str | None = None,
):
    wandb.init(
        project="simple-model-train",
        notes="version 1",
        # еще добавим возможность называть запуски
        name=run_name,
        tags=[config.optimizer_type, str(config.scheduler_type)],
        config=config,
    )

    X_val_dev = X_val.clone().to(device)
    y_val_dev = y_val.clone().to(device)

    #### Новое: создаем разный optimizer в зависимости от конфига ####
    if config.optimizer_type == "sgd":
        optim = SGD(model.parameters(), lr=config.lr)
    else:
        optim = Adam(model.parameters(), lr=config.lr)
    #### Новое: создаем шедулер ####
    # LR scheduler на вход принимает optimizer и некоторые параметры (которые зависят от его алгоритма)
    if config.scheduler_type == "exp":
        scheduler = ExponentialLR(optim, gamma=0.99)
    elif config.scheduler_type == "linear":
        scheduler = LinearLR(
            optim, start_factor=1.0, end_factor=0.1, total_iters=config.total_iterations
        )
    elif config.scheduler_type == "step":
        scheduler = StepLR(optim, step_size=10, gamma=0.9)
    else:
        scheduler = None
    #####

    model = model.to(device)
    model.train()
    for i in tqdm.trange(config.total_iterations):
        optim.zero_grad()

        X_train_dev = X_train.clone().to(device)
        y_train_dev = y_train.clone().to(device)      
        
        loss = F.cross_entropy(model(X_train_dev), y_train_dev)
        loss.backward()
        optim.step()
        metrics = {"iteration": i, "loss_train": loss.detach().cpu().item()}
        # каждые `eval_every` итераций будем считать метрику на отложенной выборке
        if (i + 1) % config.eval_every == 0:
            with torch.no_grad():
                model.eval()
                loss_val = F.cross_entropy(model(X_val_dev), y_val_dev)
                model.train()
                metrics.update({"loss_val": loss_val.cpu().item()})
        if scheduler is not None:
            # для scheduler точно так же надо звать .step(), но после обучения и валидации
            # см. пример в документации: https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html
            scheduler.step()
            metrics.update({"lr": scheduler.get_last_lr()[0]})
        else:
            # Чтобы иметь одинаковый набор графиков
            metrics.update({"lr": config.lr})
        wandb.log(metrics)
    wandb.finish()
    return optim


# на exp быстрее к нулю сойдемся, без scheduler тогда ок.
for optim in ("sgd", "adam"):
    for scheduler_type in (None, "exp", "linear", "step"):
        torch.random.manual_seed(seed)
        config = TrainConfig(
            eval_every=20,
            lr=2,
            total_iterations=500,
            scheduler_type=scheduler_type,
            optimizer_type=optim,
        )
        model = SimpleModel(num_classes=len(ohe.classes_))
        train_loop_dev(
            model,
            X_train,
            y_train,
            X_val,
            y_val,
            config=config,
            run_name=f"optim={optim}__lr_sched={scheduler_type}",
        )

#### Что изменил LR Scheduler
Добавление экспоненциального шедулера помогло в начале, но в конце привело к худшей метрике,
чем полное отсутствие шедулера.

Делаем вывод, что LR scheduler не всегда дает пользу.

В больших сетях без него не обойтись, поскольку большие нейросети бывают капризными к выбору learning rate.
Наша же сеть простая, поэтому для нее не имеет разницы - что есть шедулер, что его нет.

#### Что изменил другой оптимайзер
Смена SGD на Adam привела к ухудшению результата модели.
Делаем вывод, что менять оптимайзер - не самая лучшая идея, если хотим выбить больше качества.

Но если модель не хочет учиться, то смена оптимайзера может спасти дело.
Такое случается в больших моделях.


### Добавляем Dropout и Batch Normalization

In [ ]:
class DropoutModel(nn.Module):
    def __init__(self, num_classes: int, p_dropout: float = 0.5):
        super().__init__()
        hidden_dim = 256
        self.net = nn.Sequential(
            nn.Linear(in_features=28 * 28, out_features=hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=p_dropout),
            nn.Linear(in_features=hidden_dim, out_features=num_classes),
            nn.Softmax(dim=1),
        )

    def forward(self, x: torch.Tensor):
        x = x.reshape((-1, 28 * 28))
        return self.net(x)


torch.random.manual_seed(seed)
config = TrainConfig(
    eval_every=20,
    lr=2,
    total_iterations=500,
    # возьмем None как самый лучший вариант
    scheduler_type=None,
    optimizer_type='sgd',
)
model = DropoutModel(num_classes=len(ohe.classes_))
train_loop(model, X_train, y_train, X_val, y_val, config=config, run_name="add-dropout")

Лосс на трейне оказался чуть больше, но зато на валидации ошибка оказалась чуть-чуть получше.
Это ожидаемо: dropout нацелен на то, чтобы бороться с переобучением.
Dropout ставит палки в колеса модели, отсюда и увеличенный лосс на трейне.

In [ ]:
class BatchNormModel(nn.Module):
    def __init__(self, num_classes: int, p_dropout: float = 0.5):
        super().__init__()
        hidden_dim = 256
        self.net = nn.Sequential(
            nn.Linear(in_features=28 * 28, out_features=hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=hidden_dim),
            nn.Linear(in_features=hidden_dim, out_features=num_classes),
            nn.Softmax(dim=1),
        )

    def forward(self, x: torch.Tensor):
        x = x.reshape((-1, 28 * 28))
        return self.net(x)


torch.random.manual_seed(seed)
config = TrainConfig(
    eval_every=20,
    lr=2,
    total_iterations=500,
    scheduler_type=None,
    optimizer_type="sgd",
)
model = BatchNormModel(num_classes=len(ohe.classes_))
optim = train_loop(
    model, X_train, y_train, X_val, y_val, config=config, run_name="add-batchnorm"
)

Лосс упал и на трейне, и на валидации.

Получается, нормализация выходов слоев действительно помогает выбивать лучшие результаты.

## Как сохранить обученную модель

Отлично, у нас есть улучшение бейзлайна - это SGD + BatchNormalization.

Мы даже его уже обучили!
Можем ли мы как-то сделать так, чтобы при следующем запуске не надо было заново учить? Да, можем.

In [ ]:
# Все состояние модели хранится в .state_dict() - словаре из тензоров
model.state_dict()

In [ ]:
# Это состояние можно сохранить на диск через torch.save

# Обычно файлы pytorch сохраняют с расширением .pt (это не жесткое правило, скорее для понимания)
torch.save(model.state_dict(), "model.pt")

In [ ]:
# и потом можно загрузить

# сохраняются веса, но не объект модели - поэтому его надо создать заново
model_loaded = DropoutModel(num_classes=len(ohe.classes_))
# грузим файл, затем просим pytorch восстановить state_dict из заданного словаря
model_loaded.load_state_dict(torch.load("model.pt"))
# о нет, не получилось.

In [ ]:
# Если какие-то ключи не нашлись в файле или лишние, pytorch не даст загрузить.
# Смотрим выше и понимаем, что модель была BatchNormModel - там другой слой.
model_loaded = BatchNormModel(num_classes=len(ohe.classes_))
model_loaded.load_state_dict(torch.load("model.pt"))

In [ ]:
# Теперь все ок.
# Проверим, что веса прогрузились те же, что и были
from torch.testing import assert_close

assert_close(model_loaded(X_train), model(X_train))

### Пара слов про optimizer
Когда мы учим модель, оптимизатор тоже может хранить какие-то тензоры.

Например, Adam хранит скользящее среднее и дисперсию по всем увиденным данным.

Поэтому вместе с моделью надо сохранять и оптимизатор. Делается это точно так же, как с моделями.

In [ ]:
optim = Adam(model_loaded.parameters())
optim.state_dict()

In [ ]:
torch.save(optim.state_dict(), "optimizer.pt")
optim_loaded = Adam(model.parameters())
optim_loaded.load_state_dict(torch.load('optimizer.pt'))

Что будет плохого, если забыть про это и инициализировать с нуля оптимизатор?

У вас не прогрузится то состояние, в котором был оптимизатор на момент последней итерации обучения.
Соответственно, результаты получатся разными, если:
- обучиться 100 итераций, сохранить на диск, потом продолжить и еще 100 итераций сделать;
- сразу сделать 200 итераций;

Это нехорошо, ведь мы хотим получать одинаковые результаты в обоих случаях.
Поэтому **не забывайте сохранять состояние оптимизатора** вместе с весами модели.

## Резюме
1. Посмотрели на работу Batch Normalization и Dropout в PyTorch.
2. Познакомились с wandb и тем, как с его помощью логгировать метрики и графики.
3. Узнали, как добиваться воспроизводимости на практике.
4. Познакомились в LR Scheduler, попробовали его в эксперименте.
5. Попробовали Batch Normalization и Dropout в имеющейся сети, сравнили качество.
6. Лучшую модель сохранили на диск.